<a href="https://colab.research.google.com/github/HarriG109/Dissertation/blob/main/Predictions_BERT_OLID_NOT_OFF_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Predictions with Finalised Model


Check available GPU:

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Install transformers package:

In [ ]:
!pip install transformers

Mount Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Load the saved tokenizer:

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
output_dir = "/content/drive/MyDrive/Model/BERT Model/OFFON"
tokenizer = BertTokenizer.from_pretrained(output_dir)

Load the saved BERT model:

In [ ]:
from transformers import BertForSequenceClassification, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(output_dir)

# Tell pytorch to run this model on the GPU.
model.cuda()

Import data for running predictions:

In [ ]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("/content/drive/MyDrive/Data/Reddit Data/Fully_Clean/skyrim_comments-clean1.csv", delimiter='\t', header=None, names=['sentence'])

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

# Create sentence and label lists
sentences = df.sentence.values

#Create array of identifiers
target_names = ['NOT', 'OFF']

Create function for applying model:

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=510, return_tensors="pt").to("cuda")
    # perform inference to our model
    with torch.no_grad():
      outputs = model(**inputs)
      # get output probabilities by doing softmax
      probs = outputs[0].softmax(1)
      
    # executing argmax function to get the candidate label
    return target_names[probs.argmax()]

Loop through posts using function to get classification per post

In [ ]:
import pandas as pd
final = pd.DataFrame(columns=['sent', 'off_on'])
i = 0
for sent in sentences:
  i += 1
  if(i % 50000  == 0):
    print(i)
  to_append = [sent, str(get_prediction(sent))]
  final_ln = len(final)
  final.loc[final_ln] = to_append

Output prediction results

In [ ]:
#Write to CSV
final.to_csv('./skyrim_predictions_OFF.csv', header=False, index=False)

In [ ]:
!mv "/content/skyrim_prediction_OFF.csv" "/content/drive/MyDrive/Data/Reddit Data/Predictions_OFF"